In [ ]:
#패키지 불러오기

import pandas as pd
import numpy as np
import datetime

date 컬럼 기준으로 6개집단 평균

# window_size = 4

In [ ]:
#데이터 불러오기

train = pd.read_excel('/content/2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_댐유입량,강우,수위데이터_210902_update.xlsx', skiprows=[0])
storm = pd.read_csv('/content/storm_data.csv', encoding='cp949')

In [ ]:
#6개의 독립적인 데이터 집단을 분리하여 행으로 쌓기

data = train.copy()

data_1 = pd.concat([data.iloc[:,:6],data.iloc[:,6:13]], axis=1)
data_2 = pd.concat([data.iloc[:,:6],data.iloc[:,13:20]], axis=1)
data_3 = pd.concat([data.iloc[:,:6],data.iloc[:,20:27]], axis=1)
data_4 = pd.concat([data.iloc[:,:6],data.iloc[:,27:34]], axis=1)
data_5 = pd.concat([data.iloc[:,:6],data.iloc[:,34:41]], axis=1)
data_6 = pd.concat([data.iloc[:,:6],data.iloc[:,41:50]], axis=1)

data_1 = data_1.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})
data_2 = data_2.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})
data_3 = data_3.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})
data_4 = data_4.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})
data_5 = data_5.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})
data_6 = data_6.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})

tmp = np.append(data_1.values,data_2.values, axis=0)
tmp = np.append(tmp,  data_3.values, axis=0)
tmp = np.append(tmp, data_4.values, axis=0)
tmp = np.append(tmp ,data_5.values, axis=0)
tmp = np.append(tmp, data_6.values, axis=0)

total = pd.DataFrame(tmp, columns=data_1.columns)

In [ ]:
#시간을 나타내주는 date변수 생성
for i in range(len(total)):
    if total.iloc[i][4] == 24:
        total.iloc[i][3] +=1
        total.iloc[i][4] = 0
        if total.iloc[i][3] == 31:
            if total.iloc[i][2] in [6,9]:
                total.iloc[i][2] +=1
                total.iloc[i][3] = 1
        if total.iloc[i][3] == 32:
            if total.iloc[i][2] in [7,8]:
                total.iloc[i][2] +=1
                total.iloc[i][3] = 1

total['date'] = total.apply(lambda x:datetime.datetime(int(x[1]), int(x[2]), int(x[3]), int(x[4])), axis=1)

In [ ]:
#파생변수 생성에 관련된 전처리 코드
#위에서 행을 기준으로 쌓은 데이터를 시간 및 홍수사상을 기준으로 평균을 내서 사용

def preprocess(train, storm, 
               transform='boxcox',
               cutoff_x=0.5, cutoff_lagx=0.5,
               window_ma=3, kernel_ma=None, impute_ma='naive', window_ewma=3, impute_ewma='naive',
               impute_cs='naive') :

  #변수명 변경
  train.rename(columns={'강우(A지역)':'강우A',	'강우(B지역)':'강우B', 	'강우(C지역)':'강우C',	'강우(D지역)':'강우D',	'수위(D지역)':'수위D',	'수위(E지역)':'수위E', '유역평균강수':'평균강수'}, inplace=True)
  
  #변수 변환(유입량을 포함한 모든 수치형 데이터)
  if transform == None : pass #변수변환 하지 않는 경우

  if transform == 'log' :     #로그변환
    train.loc[:,['강우A', '강우B', '강우C', '강우D', '수위D',
       '수위E', '평균강수', '유입량']] = np.log1p(train.loc[:,['강우A', '강우B', '강우C', '강우D', '수위D',
       '수위E', '평균강수', '유입량']])
  
  if transform == 'boxcox' :  #boxcox변환
    for i in range(5, 48) :
      train.iloc[:,i] = pd.Series(stats.boxcox(train.iloc[:,i] + 1)[0])

    
  #수위E 변수 제곱
  for i in list(train.columns) :
    if 'E' in i : train[i] = train[i]**2


  X = train.pivot_table(index=['홍수사상번호', 'date'], aggfunc='mean').reset_index().loc[:,['홍수사상번호','연','월','일','시간','유입량', 'date']]

  #각 독립변수의 mean, max, standard deviation, range, IQR, skewness변수를 파생변수로 생성
  cols = ['평균강수','강우A','강우B','강우C','강우D','수위E','수위D'] 
  for col in cols:
    for method in ['mean', 'max', 'std', 'range', 'IQR', 'skew'] :
      X[f'{col}_{method}'] = 0
      if method == 'mean':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=cols, aggfunc='mean' ).values

      if method == 'max':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=col, aggfunc='max').values
      
      if method == 'std':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=col, aggfunc='std').values

      if method == 'range':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=col, aggfunc=lambda x: x.max() - x.min() ).values

      if method == 'IQR':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=col, aggfunc=lambda x: x.quantile(0.75) - x.quantile(0.25) ).values

      if method == 'skew':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=col, aggfunc='skew' ).values

  #cutoff를 설정하여 기준보다 낮은 상관계수를 가지는 변수를 제거
  X = X[['홍수사상번호','연','월','일','시간','date'] + X.corr().유입량.where(lambda x : x>=cutoff_x).dropna().index.tolist()]


  #각 홍수사상 별로 진행정도를 나타내는 index를 생성
  def add_index(group):
    group['event_idx'] = np.arange(1,len(group)+1)
    return group

  X = X.groupby('홍수사상번호', as_index=False).apply(add_index)

  #event_idx / 전체 event_idx의 평균(평균 홍수사상 길이 대비 홍수사상의 진행률을 나타냄)
  idx_mean = X.pivot_table(index='홍수사상번호', aggfunc='max', values='event_idx').mean().values
  X['event_idx_ratio'] = X['event_idx'] / idx_mean


  #7월과 나머지를 나타내는 범주형 변수 생성
  X['july'] = 0
  X.july = np.where(X.월==7, 1, 0)

  #절기를 구분하는 범주형 변수 생성(7월 7일: 소서, 7월 23일: 대서)
  X['solar_term'] = 0
  X.solar_term = np.where( ((X.월==7) & (X.일<7)) | (X.월==6), 1,
                              np.where( (X.월==7) & (X.일>=7) & (X.일<=23), 2, 3))
  


  #X의 lag변수 생성
  #lag변수를 생성할 때 생기는 결측값을 대체하는 다양한 방법 사용
  def make_lag(lag, col, df,group_col,group,imputation='naive'):
    tmp = df[df[group_col] == group]
    if f'lag_{lag}_{col}' not in df.columns:
        df[f'lag_{lag}_{col}'] = 0
    if imputation == 'zero':
        tmp[f'lag_{lag}_{col}'] =tmp[col].shift(lag).fillna(0)

    if imputation == 'naive':
        tmp[f'lag_{lag}_{col}'] =tmp[col].shift(lag).fillna(method='backfill')

    if imputation == 'frequent':
        tmp[f'lag_{lag}_{col}'] =tmp[col].shift(lag).fillna(tmp[f'lag_{lag}_{col}'].mode()[0])
        ##최빈값 여러개 나올 경우 조정 필요

    if imputation == 'mean':
        tmp[f'lag_{lag}_{col}'] =tmp[col].shift(lag).fillna(tmp[f'lag_{lag}_{col}'].mean())
    
    return tmp

  cols = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx', '홍수사상번호','event_idx_ratio'], axis=1).columns
  groups = train.홍수사상번호.unique().tolist()
  for c in cols:
    for g in groups:
      X[X['홍수사상번호'] == g] = make_lag(1, c, X, '홍수사상번호', g)

  #수치형 변수에 대한 MA(Moving Average)변수 생성
  def make_ma(col, df, group, window=window_ma, kernel=kernel_ma, imputation=impute_ma):
    tmp = df[df['홍수사상번호'] == group]
    if f'ma_{col}' not in df.columns.drop('홍수사상번호'):
      df[f'ma_{col}'] = 0
    if imputation == 'zero':
      tmp[f'ma_{col}'] =tmp[col].rolling(win_type=kernel, window = window).mean().fillna(0)
    if imputation == 'naive':
      tmp[f'ma_{col}'] =tmp[col].rolling(win_type=kernel, window = window).mean().fillna(method='backfill')
    if imputation == 'frequent':
      tmp[f'ma_{col}'] =tmp[col].rolling(win_type=kernel, window = window).mean().fillna(tmp[f'ma_{col}'].mode()[0])
    if imputation == 'mean':
      tmp[f'ma_{col}'] =tmp[col].rolling(win_type=kernel, window = window).mean().fillna(tmp[f'ma_{col}'].mean())
    return tmp

  df_ma = pd.DataFrame()

  df = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx','event_idx_ratio'], axis=1)
  cols = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx', '홍수사상번호','event_idx_ratio'], axis=1).columns
  groups = df.홍수사상번호.unique().tolist()
  for c in cols :
    for g in groups :
      df[df['홍수사상번호']==g] = make_ma(col=c, df=df, group=g)
    df_ma = pd.concat([df_ma, df], axis=1)
    df_ma = df_ma.loc[:,df_ma.columns.str.contains('ma_')]
    df_ma = df_ma.reset_index(drop=True)

  #수치형 변수에 대한 EWMA(Exponentially Weighted Moving Average)변수 생성
  def make_ewma(col, df, group, window=window_ewma, imputation=impute_ewma):
    tmp = df[df['홍수사상번호'] == group]
    if f'ewma_{col}' not in df.columns:
      df[f'ewma_{col}'] = 0
    if imputation == 'zero':
      tmp[f'ewma_{col}'] =tmp[col].ewm(window).mean().fillna(0)
    if imputation == 'naive':
      tmp[f'ewma_{col}'] =tmp[col].ewm(window).mean().fillna(method='backfill')
    if imputation == 'frequent':
      tmp[f'ewma_{col}'] =tmp[col].rolling(window).mean().fillna(tmp[f'ewma_{col}'].mode()[0])
    if imputation == 'mean':
      tmp[f'ewma_{col}'] =tmp[col].rolling(window).mean().fillna(tmp[f'ewma_{col}'].mean())
    return tmp

  df_ewma = pd.DataFrame()
  df = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx','event_idx_ratio'], axis=1)
  cols = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx', '홍수사상번호','event_idx_ratio'], axis=1).columns
  groups = df.홍수사상번호.unique().tolist()
  for c in cols :
    for g in groups :
      df[df['홍수사상번호']==g] = make_ewma(col=c, df=df, group=g)
    df_ewma = pd.concat([df_ewma, df], axis=1)
    df_ewma = df_ewma.loc[:,df_ewma.columns.str.contains('ewma_')]
  df_ewma = df_ewma.reset_index(drop=True)

  #수치형 변수에 대한 누적합 및 변화율 변수 생성
  def cum_sum_diff(col, df, group, imputation=impute_cs):
    tmp = df[df['홍수사상번호'] == group]
    if f'cumsum_{col}' not in df.columns:
      df[f'cumsum_{col}'] = 0
    tmp[f'cumsum_{col}'] = 0
    tmp[f'cumsum_{col}'] = tmp[col].cumsum()


    if f'diff_{col}' not in df.columns:
      df[f'diff_{col}'] = 0
    tmp[col].replace(0, 1, inplace=True)
    if imputation == 'zero':
      tmp[f'diff_{col}'] = (tmp[col]- tmp[col].shift(1)) / tmp[col].shift(1)
      tmp[f'diff_{col}'].fillna(0, inplace=True)
    if imputation == 'naive':
      tmp[f'diff_{col}'] = (tmp[col]- tmp[col].shift(1)) / tmp[col].shift(1)
      tmp[f'diff_{col}'].fillna(method='backfill', inplace=True)
    if imputation == 'frequent':
      tmp[f'diff_{col}'] = (tmp[col]- tmp[col].shift(1)) / tmp[col].shift(1)
      tmp[f'diff_{col}'].fillna(tmp[f'diff_{col}'].mode()[0], inplace=True)
    if imputation == 'mean':
      tmp[f'diff_{col}'] = (tmp[col]- tmp[col].shift(1)) / tmp[col].shift(1)
      tmp[f'diff_{col}'].fillna(tmp[f'diff_{col}'].mean(), inplace=True)
    return tmp

  df_cumsum = pd.DataFrame()
  df = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx','event_idx_ratio'], axis=1)
  cols = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx','event_idx_ratio', '홍수사상번호'], axis=1).columns
  groups = df.홍수사상번호.unique().tolist()
  for c in cols :
    for g in groups :
      df[df['홍수사상번호']==g] = cum_sum_diff(col=c, df=df, group=g)
    df_cumsum = pd.concat([df_cumsum, df], axis=1)
    df_cumsum = df_cumsum.loc[:,(df_cumsum.columns.str.contains('cumsum_'))|(df_cumsum.columns.str.contains('diff_'))]
  df_cumsum = df_cumsum.reset_index(drop=True)

  #MA, EWMA, Cumsum, diff변수 병합
  X = pd.concat([X, df_ma, df_ewma, df_cumsum], axis=1)
  X = X.loc[:,~X.columns.duplicated()]

  #낮과 밤을 구분하는 변수 생성
  X["day_or_not"] = np.where((X.시간>=6)&(X.시간<=20), 1, 0)
  
  #태풍의 영향 정도를 0, 1, 2로 구분하는 변수 생성
  storm = storm.loc[:,("영향도", "발생")]
  year=[] ; month=[] ; date=[]
  for i in range(len(storm)) :
    year.append(float((storm.발생[i][0:4])))
    month.append(float((storm.발생[i][6:7])))
    date.append(float((storm.발생[i][8:10])))
  storm["연"]=year
  storm["월"]=month
  storm["일"]=date
  storm = storm.loc[:,("연", "월", "일", "영향도")]

  X = pd.merge(storm, X, how="right", on=['연','월','일'])
  X = X.fillna({'영향도':0})
  X.loc[X['영향도'] == "없음", '영향도'] = 0
  X.loc[X['영향도'] == "상륙", '영향도'] = 1
  X.loc[X['영향도'] == "직접영향", '영향도'] = 2
  
  X.rename(columns = {'영향도' : 'storm_influence'}, inplace = True)
  X.storm_influence = X.storm_influence.astype("category")

  #train, validation, test, submit 데이터 나누기
  val = X.loc[X.홍수사상번호==24,:]
  test = X.loc[X.홍수사상번호==25,:]
  submit = X.loc[X.홍수사상번호==26, :]
  
  check = [val, test]
  train = X.loc[~X.홍수사상번호.isin([24,25,26]),:]

  return train, check, submit

In [ ]:
train, check, submit = preprocess(total, # raw data
                                 storm,  # storm data (directory : ./data/~_210803/storm_data.csv)
                                 transform='log', # log or boxcox
                                 cutoff_x=0.5, # mean~skew of X : only for correlation>=cutoff_x
                                 cutoff_lagx=0.5, # mean~skew of lag(X) : only for correlation>=cutoff_lagx
                                 window_ma=4, # window size of make_ma function
                                 kernel_ma=None, # kernel : gaussian etc
                                 impute_ma='naive', # missing value imputation method : mean, zero, naive, frequent
                                 window_ewma=4, # window size of make_ewma function
                                 impute_ewma='naive', # missing value imputation method : mean, zero, naive, frequent
                                 impute_cs='naive') # missing value imputation method : mean, zero, naive, frequent

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [ ]:
train.to_csv('train.csv', index=False)
check[0].to_csv('check0.csv', index=False)
check[1].to_csv('check1.csv', index=False)
submit.to_csv('submit.csv', index=False)

# window_size = 5

In [ ]:
#데이터 불러오기

train = pd.read_excel('/content/2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_댐유입량,강우,수위데이터_210902_update.xlsx', skiprows=[0])
storm = pd.read_csv('/content/storm_data.csv', encoding='cp949')

In [ ]:
#6개의 독립적인 데이터 집단을 분리하여 행으로 쌓기

data = train.copy()

data_1 = pd.concat([data.iloc[:,:6],data.iloc[:,6:13]], axis=1)
data_2 = pd.concat([data.iloc[:,:6],data.iloc[:,13:20]], axis=1)
data_3 = pd.concat([data.iloc[:,:6],data.iloc[:,20:27]], axis=1)
data_4 = pd.concat([data.iloc[:,:6],data.iloc[:,27:34]], axis=1)
data_5 = pd.concat([data.iloc[:,:6],data.iloc[:,34:41]], axis=1)
data_6 = pd.concat([data.iloc[:,:6],data.iloc[:,41:50]], axis=1)

data_1 = data_1.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})
data_2 = data_2.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})
data_3 = data_3.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})
data_4 = data_4.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})
data_5 = data_5.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})
data_6 = data_6.rename(columns={'Unnamed: 0':'홍수사상번호', 'Unnamed: 1':'연','Unnamed: 2':'월', 'Unnamed: 3':'일', 'Unnamed: 4':'시간', 'Unnamed: 5':'유입량'})

tmp = np.append(data_1.values,data_2.values, axis=0)
tmp = np.append(tmp,  data_3.values, axis=0)
tmp = np.append(tmp, data_4.values, axis=0)
tmp = np.append(tmp ,data_5.values, axis=0)
tmp = np.append(tmp, data_6.values, axis=0)

total = pd.DataFrame(tmp, columns=data_1.columns)

In [ ]:
#시간을 나타내주는 date변수 생성
for i in range(len(total)):
    if total.iloc[i][4] == 24:
        total.iloc[i][3] +=1
        total.iloc[i][4] = 0
        if total.iloc[i][3] == 31:
            if total.iloc[i][2] in [6,9]:
                total.iloc[i][2] +=1
                total.iloc[i][3] = 1
        if total.iloc[i][3] == 32:
            if total.iloc[i][2] in [7,8]:
                total.iloc[i][2] +=1
                total.iloc[i][3] = 1

total['date'] = total.apply(lambda x:datetime.datetime(int(x[1]), int(x[2]), int(x[3]), int(x[4])), axis=1)

In [ ]:
#파생변수 생성에 관련된 전처리 코드
#위에서 행을 기준으로 쌓은 데이터를 시간 및 홍수사상을 기준으로 평균을 내서 사용

def preprocess(train, storm, 
               transform='boxcox',
               cutoff_x=0.5, cutoff_lagx=0.5,
               window_ma=3, kernel_ma=None, impute_ma='naive', window_ewma=3, impute_ewma='naive',
               impute_cs='naive') :

  #변수명 변경
  train.rename(columns={'강우(A지역)':'강우A',	'강우(B지역)':'강우B', 	'강우(C지역)':'강우C',	'강우(D지역)':'강우D',	'수위(D지역)':'수위D',	'수위(E지역)':'수위E', '유역평균강수':'평균강수'}, inplace=True)
  
  #변수 변환(유입량을 포함한 모든 수치형 데이터)
  if transform == None : pass #변수변환 하지 않는 경우

  if transform == 'log' :     #로그변환
    train.loc[:,['강우A', '강우B', '강우C', '강우D', '수위D',
       '수위E', '평균강수', '유입량']] = np.log1p(train.loc[:,['강우A', '강우B', '강우C', '강우D', '수위D',
       '수위E', '평균강수', '유입량']])
  
  if transform == 'boxcox' :  #boxcox변환
    for i in range(5, 48) :
      train.iloc[:,i] = pd.Series(stats.boxcox(train.iloc[:,i] + 1)[0])

    
  #수위E 변수 제곱
  for i in list(train.columns) :
    if 'E' in i : train[i] = train[i]**2


  X = train.pivot_table(index=['홍수사상번호', 'date'], aggfunc='mean').reset_index().loc[:,['홍수사상번호','연','월','일','시간','유입량', 'date']]

  #각 독립변수의 mean, max, standard deviation, range, IQR, skewness변수를 파생변수로 생성
  cols = ['평균강수','강우A','강우B','강우C','강우D','수위E','수위D'] 
  for col in cols:
    for method in ['mean', 'max', 'std', 'range', 'IQR', 'skew'] :
      X[f'{col}_{method}'] = 0
      if method == 'mean':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=cols, aggfunc='mean' ).values

      if method == 'max':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=col, aggfunc='max').values
      
      if method == 'std':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=col, aggfunc='std').values

      if method == 'range':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=col, aggfunc=lambda x: x.max() - x.min() ).values

      if method == 'IQR':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=col, aggfunc=lambda x: x.quantile(0.75) - x.quantile(0.25) ).values

      if method == 'skew':
        X[f'{col}_{method}'] = train.pivot_table(index=['홍수사상번호','date'], values=col, aggfunc='skew' ).values

  #cutoff를 설정하여 기준보다 낮은 상관계수를 가지는 변수를 제거
  X = X[['홍수사상번호','연','월','일','시간','date'] + X.corr().유입량.where(lambda x : x>=cutoff_x).dropna().index.tolist()]


  #각 홍수사상 별로 진행정도를 나타내는 index를 생성
  def add_index(group):
    group['event_idx'] = np.arange(1,len(group)+1)
    return group

  X = X.groupby('홍수사상번호', as_index=False).apply(add_index)

  #event_idx / 전체 event_idx의 평균(평균 홍수사상 길이 대비 홍수사상의 진행률을 나타냄)
  idx_mean = X.pivot_table(index='홍수사상번호', aggfunc='max', values='event_idx').mean().values
  X['event_idx_ratio'] = X['event_idx'] / idx_mean


  #7월과 나머지를 나타내는 범주형 변수 생성
  X['july'] = 0
  X.july = np.where(X.월==7, 1, 0)

  #절기를 구분하는 범주형 변수 생성(7월 7일: 소서, 7월 23일: 대서)
  X['solar_term'] = 0
  X.solar_term = np.where( ((X.월==7) & (X.일<7)) | (X.월==6), 1,
                              np.where( (X.월==7) & (X.일>=7) & (X.일<=23), 2, 3))
  


  #X의 lag변수 생성
  #lag변수를 생성할 때 생기는 결측값을 대체하는 다양한 방법 사용
  def make_lag(lag, col, df,group_col,group,imputation='naive'):
    tmp = df[df[group_col] == group]
    if f'lag_{lag}_{col}' not in df.columns:
        df[f'lag_{lag}_{col}'] = 0
    if imputation == 'zero':
        tmp[f'lag_{lag}_{col}'] =tmp[col].shift(lag).fillna(0)

    if imputation == 'naive':
        tmp[f'lag_{lag}_{col}'] =tmp[col].shift(lag).fillna(method='backfill')

    if imputation == 'frequent':
        tmp[f'lag_{lag}_{col}'] =tmp[col].shift(lag).fillna(tmp[f'lag_{lag}_{col}'].mode()[0])
        ##최빈값 여러개 나올 경우 조정 필요

    if imputation == 'mean':
        tmp[f'lag_{lag}_{col}'] =tmp[col].shift(lag).fillna(tmp[f'lag_{lag}_{col}'].mean())
    
    return tmp

  cols = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx', '홍수사상번호','event_idx_ratio'], axis=1).columns
  groups = train.홍수사상번호.unique().tolist()
  for c in cols:
    for g in groups:
      X[X['홍수사상번호'] == g] = make_lag(1, c, X, '홍수사상번호', g)

  #수치형 변수에 대한 MA(Moving Average)변수 생성
  def make_ma(col, df, group, window=window_ma, kernel=kernel_ma, imputation=impute_ma):
    tmp = df[df['홍수사상번호'] == group]
    if f'ma_{col}' not in df.columns.drop('홍수사상번호'):
      df[f'ma_{col}'] = 0
    if imputation == 'zero':
      tmp[f'ma_{col}'] =tmp[col].rolling(win_type=kernel, window = window).mean().fillna(0)
    if imputation == 'naive':
      tmp[f'ma_{col}'] =tmp[col].rolling(win_type=kernel, window = window).mean().fillna(method='backfill')
    if imputation == 'frequent':
      tmp[f'ma_{col}'] =tmp[col].rolling(win_type=kernel, window = window).mean().fillna(tmp[f'ma_{col}'].mode()[0])
    if imputation == 'mean':
      tmp[f'ma_{col}'] =tmp[col].rolling(win_type=kernel, window = window).mean().fillna(tmp[f'ma_{col}'].mean())
    return tmp

  df_ma = pd.DataFrame()

  df = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx','event_idx_ratio'], axis=1)
  cols = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx', '홍수사상번호','event_idx_ratio'], axis=1).columns
  groups = df.홍수사상번호.unique().tolist()
  for c in cols :
    for g in groups :
      df[df['홍수사상번호']==g] = make_ma(col=c, df=df, group=g)
    df_ma = pd.concat([df_ma, df], axis=1)
    df_ma = df_ma.loc[:,df_ma.columns.str.contains('ma_')]
    df_ma = df_ma.reset_index(drop=True)

  #수치형 변수에 대한 EWMA(Exponentially Weighted Moving Average)변수 생성
  def make_ewma(col, df, group, window=window_ewma, imputation=impute_ewma):
    tmp = df[df['홍수사상번호'] == group]
    if f'ewma_{col}' not in df.columns:
      df[f'ewma_{col}'] = 0
    if imputation == 'zero':
      tmp[f'ewma_{col}'] =tmp[col].ewm(window).mean().fillna(0)
    if imputation == 'naive':
      tmp[f'ewma_{col}'] =tmp[col].ewm(window).mean().fillna(method='backfill')
    if imputation == 'frequent':
      tmp[f'ewma_{col}'] =tmp[col].rolling(window).mean().fillna(tmp[f'ewma_{col}'].mode()[0])
    if imputation == 'mean':
      tmp[f'ewma_{col}'] =tmp[col].rolling(window).mean().fillna(tmp[f'ewma_{col}'].mean())
    return tmp

  df_ewma = pd.DataFrame()
  df = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx','event_idx_ratio'], axis=1)
  cols = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx', '홍수사상번호','event_idx_ratio'], axis=1).columns
  groups = df.홍수사상번호.unique().tolist()
  for c in cols :
    for g in groups :
      df[df['홍수사상번호']==g] = make_ewma(col=c, df=df, group=g)
    df_ewma = pd.concat([df_ewma, df], axis=1)
    df_ewma = df_ewma.loc[:,df_ewma.columns.str.contains('ewma_')]
  df_ewma = df_ewma.reset_index(drop=True)

  #수치형 변수에 대한 누적합 및 변화율 변수 생성
  def cum_sum_diff(col, df, group, imputation=impute_cs):
    tmp = df[df['홍수사상번호'] == group]
    if f'cumsum_{col}' not in df.columns:
      df[f'cumsum_{col}'] = 0
    tmp[f'cumsum_{col}'] = 0
    tmp[f'cumsum_{col}'] = tmp[col].cumsum()


    if f'diff_{col}' not in df.columns:
      df[f'diff_{col}'] = 0
    tmp[col].replace(0, 1, inplace=True)
    if imputation == 'zero':
      tmp[f'diff_{col}'] = (tmp[col]- tmp[col].shift(1)) / tmp[col].shift(1)
      tmp[f'diff_{col}'].fillna(0, inplace=True)
    if imputation == 'naive':
      tmp[f'diff_{col}'] = (tmp[col]- tmp[col].shift(1)) / tmp[col].shift(1)
      tmp[f'diff_{col}'].fillna(method='backfill', inplace=True)
    if imputation == 'frequent':
      tmp[f'diff_{col}'] = (tmp[col]- tmp[col].shift(1)) / tmp[col].shift(1)
      tmp[f'diff_{col}'].fillna(tmp[f'diff_{col}'].mode()[0], inplace=True)
    if imputation == 'mean':
      tmp[f'diff_{col}'] = (tmp[col]- tmp[col].shift(1)) / tmp[col].shift(1)
      tmp[f'diff_{col}'].fillna(tmp[f'diff_{col}'].mean(), inplace=True)
    return tmp

  df_cumsum = pd.DataFrame()
  df = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx','event_idx_ratio'], axis=1)
  cols = X.drop(['연', '월', '일','시간', 'date','유입량', 'solar_term', 'july', 'event_idx','event_idx_ratio', '홍수사상번호'], axis=1).columns
  groups = df.홍수사상번호.unique().tolist()
  for c in cols :
    for g in groups :
      df[df['홍수사상번호']==g] = cum_sum_diff(col=c, df=df, group=g)
    df_cumsum = pd.concat([df_cumsum, df], axis=1)
    df_cumsum = df_cumsum.loc[:,(df_cumsum.columns.str.contains('cumsum_'))|(df_cumsum.columns.str.contains('diff_'))]
  df_cumsum = df_cumsum.reset_index(drop=True)

  #MA, EWMA, Cumsum, diff변수 병합
  X = pd.concat([X, df_ma, df_ewma, df_cumsum], axis=1)
  X = X.loc[:,~X.columns.duplicated()]

  #낮과 밤을 구분하는 변수 생성
  X["day_or_not"] = np.where((X.시간>=6)&(X.시간<=20), 1, 0)
  
  #태풍의 영향 정도를 0, 1, 2로 구분하는 변수 생성
  storm = storm.loc[:,("영향도", "발생")]
  year=[] ; month=[] ; date=[]
  for i in range(len(storm)) :
    year.append(float((storm.발생[i][0:4])))
    month.append(float((storm.발생[i][6:7])))
    date.append(float((storm.발생[i][8:10])))
  storm["연"]=year
  storm["월"]=month
  storm["일"]=date
  storm = storm.loc[:,("연", "월", "일", "영향도")]

  X = pd.merge(storm, X, how="right", on=['연','월','일'])
  X = X.fillna({'영향도':0})
  X.loc[X['영향도'] == "없음", '영향도'] = 0
  X.loc[X['영향도'] == "상륙", '영향도'] = 1
  X.loc[X['영향도'] == "직접영향", '영향도'] = 2
  
  X.rename(columns = {'영향도' : 'storm_influence'}, inplace = True)
  X.storm_influence = X.storm_influence.astype("category")

  #train, validation, test, submit 데이터 나누기
  val = X.loc[X.홍수사상번호==24,:]
  test = X.loc[X.홍수사상번호==25,:]
  submit = X.loc[X.홍수사상번호==26, :]
  
  check = [val, test]
  train = X.loc[~X.홍수사상번호.isin([24,25,26]),:]

  return train, check, submit

In [ ]:
train, check, submit = preprocess(total, # raw data
                                 storm,  # storm data (directory : ./data/~_210803/storm_data.csv)
                                 transform='log', # log or boxcox
                                 cutoff_x=0.5, # mean~skew of X : only for correlation>=cutoff_x
                                 cutoff_lagx=0.5, # mean~skew of lag(X) : only for correlation>=cutoff_lagx
                                 window_ma=5, # window size of make_ma function
                                 kernel_ma=None, # kernel : gaussian etc
                                 impute_ma='naive', # missing value imputation method : mean, zero, naive, frequent
                                 window_ewma=5, # window size of make_ewma function
                                 impute_ewma='naive', # missing value imputation method : mean, zero, naive, frequent
                                 impute_cs='naive') # missing value imputation method : mean, zero, naive, frequent

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [ ]:
train.to_csv('train_std_5.csv', index=False)
check[0].to_csv('check0_std_5.csv', index=False)
check[1].to_csv('check1_std_5.csv', index=False)
submit.to_csv('submit_std_5.csv', index=False)